<center>
    <b> Microproyecto #7 </b>
    <br/> Javier Anleu 17149
    <br/> Andrea Argüello 17801
</center>

# Función inversa de distribución exponencial, E[X] y Var(X)

In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
from IPython.display import display, Latex

In [2]:
exponentialInv = lambda x,lam: -np.log(1-x)/lam # donde x es U(0,1)
mu_exponencial = lambda lam: 1/lam
sigma_cuadrado_exponencial = lambda lam: 1/(lam**2)

## Ejercicio 1: Variables antitéticas, $\lambda = 1$ 

In [3]:
lambda_1 = 1

In [4]:
random.seed(500)
def ej1(repeticiones):
    X1 = []
    X2 = []
    Y = []
    for i in range(repeticiones):
        x = 1
        while x == 1:
            x = random.uniform(0,1)
        X1.append(exponentialInv(x,lambda_1))
        X2.append(exponentialInv(1-x,lambda_1))
        Y.append((X1[-1]+X2[-1])/2)
    return X1, X2, Y

In [5]:
cov_xy = lambda X, Y: np.mean([x * y for x, y in zip(X, Y)])-np.mean(X)*np.mean(Y)

In [6]:
var = lambda X: np.mean([x**2 for x in X])-(np.mean(X))**2

In [7]:
var_mean_xy = lambda X, Y: (var(X) + var(Y) + 2*cov_xy(X,Y))/4

In [24]:
X1, X2, Y = ej1(10000)

In [25]:
display(Latex(f'$Cov(X_1,X_2)$ = %f' % (cov_xy(X1,X2))))

<IPython.core.display.Latex object>

Queda demostrado que la covarianza entre $X_1$ y $X_2$ es negativa. Como ambas provienen de la distribución exponencial, entonces $X_1$ y $X_2$ son antitéticas.

In [26]:
display(Latex(f'$E[Y]$ obtenido: %f, $\mu$ esperado: %f' % (np.mean(Y), mu_exponencial(lambda_1))))

<IPython.core.display.Latex object>

In [27]:
display(Latex(f'$Var(Y)$ obtenida: %f, $\sigma^2$ esperada: %f' % (var(Y), sigma_cuadrado_exponencial(lambda_1))))

<IPython.core.display.Latex object>

## Ejercicio 2: Variables de control, $\lambda = 1$ 

In [28]:
random.seed(487)
def ej2(repeticiones):
    X = []
    Y = []
    for i in range(repeticiones):
        x = 1
        y = 1
        while x == 1:
            x = random.uniform(0,1)
        while y == 1:
            y = random.uniform(0,1) # sabemos que mu_y = 0.5 y var_y = 1/12
        X.append(exponentialInv(x,lambda_1))
        Y.append(y)
    return X, Y

def c_asterisco(lista1,lista2): #Basado en el ejemplo visto en clase
    n=0
    d=0
    promedio_x=np.mean(lista1)
    promedio_y=np.mean(lista2)
    for i in range(len(lista1)):
        n=n+(lista1[i]-promedio_x)*(lista2[i]-promedio_y)
        d=d+pow(lista2[i]-promedio_y,2)
    return -1*n/d
        
# A esta, le pasamos el valor teórico de la media de la distribución conocida Y, mu_y
control = lambda X, Y, mu_y, c: [x + c*(y - mu_y) for x, y in zip(X,Y)]

X, Y = ej2(10000)

c_min = c_asterisco(X, Y)

In [29]:
display(Latex(f'$E[X]$ obtenido: %f, $\mu$ esperado: %f' % (np.mean(control(X, Y, 0.5, c_min)), mu_exponencial(lambda_1))))

<IPython.core.display.Latex object>

In [30]:
display(Latex(f'$Var(X)$ obtenida: %f, $\sigma^2$ esperada: %f' % (var(control(X, Y, 0.5, c_min)), sigma_cuadrado_exponencial(lambda_1))))

<IPython.core.display.Latex object>

## Ejercicio 3: Método estratificado, $\lambda = 1$ 

In [ ]:
bernoulliInv = lambda x,p: 1 if x>(1-p) else 0

In [ ]:
random.seed(487)
p=0.5
def ej3(repeticiones):
    X = []
    Y = []
    for i in range(repeticiones):
        x = 1
        while x == 1:
            x = random.uniform(0,1)
        X.append(exponentialInv(x,lambda_1))
        Y.append(bernoulliInv(x,p)) #idk what to do here fam sorry
    return X, Y

## Ejercicio 4

### Sea X variable aleatoria exponencial con media 1; y dado X=x, Y es una variable aleatoria exponencial con media x (X y Y son variables aleatorias dependientes). Dé una forma eficiente para estimar P(XY≤3)

Nótese que si $E[X]=1$, $\lambda_X = 1$ y si $E[Y|X=x]=x$, entonces $\lambda_Y = 1/x$.

Nota: ¿$P(XY\leq3) = P(Y \leq \frac{3}{x} | X=x)$?

In [50]:
def ej4(repeticionesx, repeticionesy):
    X=[]
    Y=[]
    for i in range(repeticionesx):
        ys = []
        x=1
        while x==1:
            x=random.uniform(0,1)
        X.append(exponentialInv(x,lambda_1))
        for i in range(repeticionesy):
            y=1
            while y==1:
                y=random.uniform(0,1)
            ys.append(exponentialInv(y,1/X[-1]))
        Y.append(ys)
    return X,Y

In [51]:
rangex, rangey = 1000,1000
X, Y = ej4(rangex,rangey)

In [52]:
total = 0
for x in range(rangex):
    for y in Y[x]:
        if X[x]*y <= 3:
            total+=1
prob = total/(rangex*rangey)

In [53]:
prob

0.860055

In [54]:
# Reduzcamos varianza
def antitetica(repeticiones, lambda_): #basada en ej1
    X1 = []
    X2 = []
    Y = []
    for i in range(repeticiones):
        x = 1
        while x == 1:
            x = random.uniform(0,1)
        X1.append(exponentialInv(x,lambda_))
        X2.append(exponentialInv(1-x,lambda_))
        Y.append((X1[-1]+X2[-1])/2)
    return Y

def ej4alt(repeticionesx, repeticionesy):
    X=antitetica(repeticionesx, 1)
    Y=[]
    for i in range(repeticionesx):
        y=antitetica(repeticionesy,1/X[i])
        Y.append(y)
    return X,Y

X, Y = ej4alt(rangex,rangey)
total = 0
for x in range(rangex):
    for y in Y[x]:
        if X[x]*y <= 3:
            total+=1
prob = total/(rangex*rangey)
prob

0.927279